$$
z_{ij} = 
\begin{Bmatrix}
1, \text{if the ith facility is assigned to the jth center} \\
0, \text{otherwise}
\end{Bmatrix}
$$

$$
y_j 
= 
\begin{Bmatrix}
1, \text{if jth location is selected as the location for a depot} \\
0, \text{otherwise}
\end{Bmatrix}
$$

$$
d_{ij} = \text{Distance between ith and jth location}
$$

$$
a_{i} = \text{Weight (demand) of the ith location}
$$

$$
\begin{aligned}
\min z = & \sum_{i=1}^{n} \sum_{j=1}^n a_i d_{ij} z_{ij} \\
\text{Subject to:} & \\
& \sum_{j = 1}^{n} z_{ij = 1}  \;\;\;, \forall_i, \;\;\; i, j = 1,2,\ldots, n \\
& z_{ij} \le y_j  \;\;\;, \forall_{i, j}\;\;\; i, j = 1,2,\ldots, n \\
& \sum_{j=1}^n y_j = 1 \\
& a_{ij}, y_j \in \{0,1\} \;\;\; i, j = 1,2,\ldots, n \\
\end{aligned}
$$

In [2]:
using JuMP, HiGHS

In [32]:
number_of_depots = 2

2

In [3]:
coords = Float64[
    1 5;
    2 4;
    4 3;
    2 1;
    7 8;
    8 7
]

6×2 Matrix{Float64}:
 1.0  5.0
 2.0  4.0
 4.0  3.0
 2.0  1.0
 7.0  8.0
 8.0  7.0

In [4]:
function euclidean(u, v)
    return ((u .- v) .* (u .- v)) |> sum |> sqrt
end

euclidean (generic function with 1 method)

In [5]:
n, p = size(coords)

(6, 2)

In [6]:
distances = zeros(Float64, n, n)

6×6 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0

In [8]:
for i in 1:n
    for j in i:n
        d = euclidean(coords[i, :], coords[j, :])
        distances[i, j] = d
        distances[j, i] = d
    end 
end 

In [9]:
distances

6×6 Matrix{Float64}:
 0.0      1.41421  3.60555  4.12311  6.7082   7.28011
 1.41421  0.0      2.23607  3.0      6.40312  6.7082
 3.60555  2.23607  0.0      2.82843  5.83095  5.65685
 4.12311  3.0      2.82843  0.0      8.60233  8.48528
 6.7082   6.40312  5.83095  8.60233  0.0      1.41421
 7.28011  6.7082   5.65685  8.48528  1.41421  0.0

In [33]:
m = Model(HiGHS.Optimizer)

@variable(m, z[1:n, 1:n], Bin)
@variable(m, y[1:n], Bin)

@constraint(m, sum(y) == number_of_depots)

for i in 1:n
    for j in 1:n
        @constraint(m, z[i, j] .<= y[j])
    end 
end

for i in 1:n
    @constraint(m, sum(z[i, :]) == 1)
end 

@objective(m, Min, sum(distances[1:n, 1:n] .* z[1:n, 1:n]))

optimize!(m)

value.(y)

Running HiGHS 1.5.1 [date: 1970-01-01, git hash: 93f1876e4]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
43 rows, 42 cols, 114 nonzeros
43 rows, 42 cols, 114 nonzeros

Solving MIP model with:
   43 rows
   42 cols (42 binary, 0 integer, 0 implied int., 0 continuous)
   114 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.0s
 T       0       0         0   0.00%   0               8.064495102      100.00%        0      0      0        17     0.0s

Solving report
  Status            Optimal
  Primal bound      8.06449510225
  Dual bound        8.06449510225
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    8.0

6-element Vector{Float64}:
 -0.0
  1.0
  0.0
 -0.0
  0.0
  1.0

In [34]:
value.(z)

6×6 Matrix{Float64}:
 -0.0  1.0   0.0   0.0   0.0  0.0
 -0.0  1.0   0.0   0.0   0.0  0.0
  0.0  1.0  -0.0  -0.0   0.0  0.0
  0.0  1.0  -0.0  -0.0   0.0  0.0
  0.0  0.0   0.0   0.0  -0.0  1.0
  0.0  0.0   0.0   0.0  -0.0  1.0

In [37]:
location_of_depots = findall(x -> x == 1, value.(y))

2-element Vector{Int64}:
 2
 6